In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory of src to the path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.db.manager import DBManager
from src.input_to_instructions.load_and_execute import *
from src.input_to_instructions.types import *
from src.plot_graph.execute import *
from src.operation.execute import *

In [3]:
from collections import defaultdict, Counter
import logging

import pandas as pd
import numpy as np
from tqdm import tqdm
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

import json
import itertools

from db.manager import DBManager
from operation.execute import OperationExecutor
from pathlib import Path
import warnings
import datetime


warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

INFO:db.instance:Connected to the database PerSite_DB


In [4]:
BASE_DIR = "../"
def read_json(path):
    with open(path, "r", encoding="utf-8") as f:
        result = json.loads(f.read())
    
    # result = [{"Input": d["Input"], "Response": json.dumps(d["Response"], ensure_ascii=False)} for d in result]
    return result

In [5]:
import matplotlib.pyplot as plt
import matplotlib

from src.plot_graph.execute import plot_graph_plotly
matplotlib.rcParams['font.family'] = 'NanumGothicCoding'
def run_query(user_input, metadata, instructions):
    variables = {
        "Metadata": metadata,
    }
    for instruction in instructions:
        # logger.debug(f"Executing instruction: {instruction.__class__.__name__}")
        # print(f"Executing instruction: {instruction.__class__.__name__}")
        
        if type(instruction) == InstructionQ:
            # Execute query
            result_df = DBManager.structured_query_data_t(metadata, instruction.args)
            
            if result_df is None:
                print("죄송합니다, 관련 데이터를 찾을 수 없습니다.", "response")
                return

            # For demo, drop rows where any value is -1
            result_df = result_df.loc[(result_df != -1).all(axis=1)]
           
            #pd.set_option('display.max_rows', 10000)        
            #pd.set_option('display.max_columns', 1000)
            #pd.set_option('display.width', 1000)
            #pd.set_option('display.max_colwidth', 1000)
            #print(f"QueryResult: {result_df}")

            variables[instruction.result_name] = result_df
        
        elif type(instruction) == InstructionO:
            # Execute operation

            result_dict = OperationExecutor.execute(variables, instruction.scripts, instruction.returns)
            
            variables.update(result_dict)
            pass
        elif type(instruction) == InstructionG:

            # fig = plot_graph(instruction, variables)
            # plt.show(fig)
            
            #fig = plot_graph_plotly(instruction, variables)
            #plt.show(fig)
            #fig.show()
            #print(type(fig))
            # print(fig, "graph")
            pass
        elif type(instruction) == InstructionR:
            #pass
            # # Execute response generation
            variables_to_report = {k: v for k, v in variables.items() if k not in ["Metadata"]}
            print(f"Variables: {variables_to_report}")
         
            
            
            # response, required_variables = ResponseGeneration.execute(instruction, variables, user_input, metadata)
            # print(f"Required variables: {required_variables}")
            
            # print(response, "response")

In [ ]:
def build_query_groundtruth(dateset_name):
    def read(path):
        data = read_json(path)
        for i, d in enumerate(data):
            data[i]["Scenario"] = directory.name
            if "v7" in dateset_name:
                data[i]["Metadata"] = metadata
        return data

    ds_ts = []
    ds_tr = []
    base_dataset_dir = Path(f"{BASE_DIR}/finetuning/dataset/{dateset_name}")
    
    for directory in base_dataset_dir.iterdir():
        if directory.is_dir() and "scenario2" in directory.name:
            if "v7" in dateset_name:
                metadata = read_json(f"{directory}/metadata.json")
                
            
            #ds_ts.extend(read(f"{directory}/onlyq_tr.json"))
            ds_tr.extend(read(f"{directory}/onlyq_ts.json"))
            #ds_tr.extend(read(f"{directory}/graph_temp.json"))
    
    ds = ds_ts + ds_tr
    
    # if "v7" in dateset_name:
    #     db_gt_filename = f"{BASE_DIR}/experiments/db_gt_v7.json"
    # else:
    #     db_gt_filename = f"{BASE_DIR}/experiments/db_gt.json"
    #     metadata = None
    
    # with open(db_gt_filename, "w", encoding="utf-8") as f:
        # f.write("[")
    # with tqdm(total=len(ds)) as pbar:
    #count=0
    for d in ds:
        #count += 1
        #if count < 10:
        #    continue  # 10보다 작은 값은 건너뛰기
        #if count > 40:
        #    break     # 20을 초과하면 반복 종료
        # pbar.set_description(f"Processing {d['Input']}")
        # print("--")
        
        instructions = InputToInstruction.postprocess(d['Response'])
        user_input, tags, metadata, scenario = d["Input"], d["Tags"], d["Metadata"], d["Scenario"]
        print(user_input)
        
        run_query(user_input, metadata, instructions)
        
  

In [21]:

build_query_groundtruth("v7-250309-reduceinputanddatefunctioncall")

오늘 4층의 평균 실내온도 알려줘
Variables: {'qr':       roomtemp           timestamp     idu
0         28.5 2022-09-17 00:00:00  02_I84
1         28.5 2022-09-17 00:01:00  02_I84
2         28.5 2022-09-17 00:02:00  02_I84
3         28.5 2022-09-17 00:03:00  02_I84
4         28.5 2022-09-17 00:04:00  02_I84
...        ...                 ...     ...
4255      25.5 2022-09-17 23:55:00  01_IB7
4256      25.5 2022-09-17 23:56:00  01_IB7
4257      25.5 2022-09-17 23:57:00  01_IB7
4258      25.5 2022-09-17 23:58:00  01_IB7
4259      25.5 2022-09-17 23:59:00  01_IB7

[4260 rows x 3 columns], 'roomtemp_avg': 27.463849765258217}
4층에서 에어컨 꺼진 방들 알려줘
Variables: {'qr':      oper           timestamp     idu
0   False 2022-09-17 16:25:00  02_I84
1   False 2022-09-17 16:26:00  02_I84
2   False 2022-09-17 16:27:00  02_I84
3   False 2022-09-17 16:28:00  02_I84
4   False 2022-09-17 16:29:00  02_I84
5   False 2022-09-17 16:30:00  02_I84
6   False 2022-09-17 16:25:00  02_I85
7   False 2022-09-17 16:26:00  02_I85
8   Fa

In [22]:
# run_scenarios.py
# -*- coding: utf-8 -*-
import json, textwrap, re
from pathlib import Path
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

BASE_DIR     = "/workspace"
dateset_name = "v7-250309-reduceinputanddatefunctioncall"

DATE_FIXES = {
    'CURRENT_DATE':      "DATE '2022-09-30'",
    'CURRENT_TIMESTAMP': "TIMESTAMP '2022-09-30 12:00:00'"
}

# ──────────────────────────────────────────
# ① 문자열 -or-리스트 모두 처리하도록 수정
def apply_date_fixes(sql):
    if isinstance(sql, list):                           # ### (수정)
        return [apply_date_fixes(s) for s in sql]       # 재귀적으로 각 요소 변환
    for k, v in DATE_FIXES.items():
        sql = re.sub(rf'\b{k}\b', v, sql)
    return sql

def wrap_into_select(sql_o: str, sql_q: str) -> str:
    if '/* SQL_Q */' in sql_o:
        sql_o = sql_o.replace('/* SQL_Q */',
                              textwrap.dedent(sql_q).rstrip(';'))
    sql_o = textwrap.dedent(sql_o).strip().rstrip(';')
    if not sql_o.lstrip().upper().startswith('SELECT'):
        sql_o = f"SELECT * FROM (\n{sql_o}\n) AS sub"
    return sql_o + ';'

def read_json(p: Path):
    with open(p, encoding='utf-8') as f:
        return json.load(f)

# ───────────────── 데이터 적재 ──────────────────
base_dataset_dir = Path(f"{BASE_DIR}/finetuning/dataset/{dateset_name}")
scenarios = []
for directory in base_dataset_dir.iterdir():
    if directory.is_dir() and "scenario2" in directory.name:
        if "v7" in dateset_name:
            _ = read_json(directory / "metadata.json")
        scenarios.extend(read_json(directory / "sql.json"))

# ───────────────── 실행 루프 ────────────────────
for idx, block in enumerate(scenarios, 1):
    inp, tags = block['Input'], block['Tags']
    sql_q_raw       = tags['SQL_Q']
    result_name_raw = tags['result_name']

    sql_q = apply_date_fixes(sql_q_raw)                 # 리스트/문자열 모두 OK
    sql_o = apply_date_fixes(tags['SQL_O'])

    print(f"\n▶ ({idx}) {inp}")

    # ---------- ① SQL_Q 실행 ----------
    # ② 리스트라면 각 문장을 따로 실행하도록 수정
    if isinstance(sql_q, list):                         # ### (수정)
        for q_idx, (single_sql, res_name) in enumerate(zip(sql_q, result_name_raw), 1):
            try:
                rows = DBManager.execute_sql(single_sql)
                desc = DBManager.db_instance.cursor.description
                #print(f"\n― SQL_Q[{q_idx}] → {res_name} ―")
                if desc:
                    cols = [d[0] for d in desc]
                    df   = pd.DataFrame(rows, columns=cols)
                    print(df if not df.empty else '(empty result)')
                else:
                    print("(non-select statement)")
            except Exception as e:
                print(f"SQL_Q[{q_idx}] ERROR → {e}")
        # wrap_into_select에서는 첫번째 쿼리만 매크로 치환에 쓰면 충분
        sql_q_for_macro = sql_q[0]
    else:                                               # 기존 단일 문자열 처리
        try:
            rows_q = DBManager.execute_sql(sql_q)
            desc_q = DBManager.db_instance.cursor.description
            if desc_q:
                cols_q = [d[0] for d in desc_q]
                df_q   = pd.DataFrame(rows_q, columns=cols_q)
                print(df_q if not df_q.empty else '(empty result)')
            else:
                print("(non-select statement)")
        except Exception as e:
            print(f"SQL_Q ERROR → {e}")
        sql_q_for_macro = sql_q

    # ---------- ② SQL_O 실행 ----------
    try:
        final_sql = wrap_into_select(sql_o, sql_q_for_macro)   # ### (수정)
        rows_o    = DBManager.execute_structured_query_string(final_sql)
        desc_o    = DBManager.db_instance.cursor.description
        if desc_o:
            cols_o = [d[0] for d in desc_o]
            df_o   = pd.DataFrame(rows_o, columns=cols_o)
            print("\n― SQL_O 결과 ―")
            print(df_o if not df_o.empty else '(empty result)')
        else:
            print(f"\n― SQL_O 수행 완료 (행 {DBManager.db_instance.cursor.rowcount}개, 반환 컬럼 없음) ―")
    except Exception as e:
        print(f"SQL_O ERROR → {e}")

DBManager.db_instance.close()


INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched



▶ (1) 오늘 4층의 평균 실내온도 알려줘
      roomtemp idu_name           timestamp
0         29.0   02_I85 2022-09-17 00:00:00
1         25.0   01_IB7 2022-09-17 00:00:00
2         28.5   02_I84 2022-09-17 00:00:00
3         29.0   02_I85 2022-09-17 00:01:00
4         25.0   01_IB7 2022-09-17 00:01:00
...        ...      ...                 ...
4255      25.5   01_IB7 2022-09-17 23:58:00
4256      26.5   02_I84 2022-09-17 23:58:00
4257      25.0   02_I85 2022-09-17 23:59:00
4258      25.5   01_IB7 2022-09-17 23:59:00
4259      26.5   02_I84 2022-09-17 23:59:00

[4260 rows x 3 columns]

― SQL_O 결과 ―
   roomtemp_avg
0      27.46385

▶ (2) 4층에서 에어컨 꺼진 방들 알려줘
     oper idu_name           timestamp
0   False   01_IB7 2022-09-17 16:25:00
1   False   02_I84 2022-09-17 16:25:00
2   False   02_I85 2022-09-17 16:25:00
3   False   01_IB7 2022-09-17 16:26:00
4   False   02_I84 2022-09-17 16:26:00
5   False   02_I85 2022-09-17 16:26:00
6   False   01_IB7 2022-09-17 16:27:00
7   False   02_I84 2022-09-17 16:27:0

INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched


        settemp idu_name           timestamp
0          23.0   02_I85 2022-08-17 00:00:00
1          23.0   02_I84 2022-08-17 00:00:00
2          23.0   01_IB7 2022-08-17 00:00:00
3          23.0   02_I85 2022-08-17 00:01:00
4          23.0   02_I84 2022-08-17 00:01:00
...         ...      ...                 ...
131941     23.0   01_IB7 2022-09-16 23:59:00
131942     23.0   02_I84 2022-09-16 23:59:00
131943     23.0   02_I85 2022-09-17 00:00:00
131944     23.0   02_I84 2022-09-17 00:00:00
131945     23.0   01_IB7 2022-09-17 00:00:00

[131946 rows x 3 columns]

― SQL_O 결과 ―
   settemp_max  settemp_min
0         25.0         21.0

▶ (5) 오늘 4층에서 가장 더운 방은 어디야?
      roomtemp idu_name           timestamp
0         29.0   02_I85 2022-09-17 00:00:00
1         25.0   01_IB7 2022-09-17 00:00:00
2         28.5   02_I84 2022-09-17 00:00:00
3         29.0   02_I85 2022-09-17 00:01:00
4         25.0   01_IB7 2022-09-17 00:01:00
...        ...      ...                 ...
4255      25.5   01_IB7 20